In [ ]:
import torch
import matplotlib.pyplot as plt
import torch.optim as optim

In [ ]:
t_c = [0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [ ]:
def model(input, w, b):
    return w * input + b

def lossFn(y_pred, y_test):
    return ((y_test-y_pred)**2).mean()

def gradFn(x, y, w, b):

    y_pred = model(x,w,b)

    dw = 2 * (y_pred-y) * x / y_pred.size(0)
    db = 2 * (y_pred-y) * 1 / y_pred.size(0)

    return torch.stack([dw.sum(), db.sum()])


def training_loop_manual(n_epochs, learning_rate, params, t_u, t_c):

    loss_mat=[]

    for epoch in range(1, n_epochs + 1):

        if epoch==30:
            learning_rate*=0.5
        elif epoch==60:
            learning_rate*=0.5

        w, b = params

        t_p = model(t_u, *params)

        loss = lossFn(t_p, t_c)

        grad = gradFn(t_u, t_c,params)
        
        loss_mat.append(loss)
        params = params - learning_rate * grad
        print('Epoch %d, Loss %f' % (epoch, float(loss)))

    return params, loss_mat


def training_loop_grad(n_epochs, learning_rate, params, t_u, t_c):

    loss_mat=[]

    for epoch in range(1, n_epochs + 1):

        if params.grad is not None:
            params.grad.zero_()

        if epoch==30:
            learning_rate*=0.5
        elif epoch==60:
            learning_rate*=0.5

        loss = lossFn(model(t_u, *params), t_c)
        loss.backward()

        # print(loss.item())

        loss_mat.append(loss.item())
        
        with torch.no_grad():
            params -= learning_rate * params.grad
            
        print('Epoch %d, Loss %f' % (epoch, float(loss)))

    return params, loss_mat




In [ ]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
lr = 1e-1
# print(params.requires_grad)
t_u2 = (t_u - t_u.mean()) / t_u.std()

if not params.requires_grad:
    params, loss_mat = training_loop_manual(
        n_epochs = 100,
        learning_rate = lr,
        params = params,
        t_u = t_u2,
        t_c = t_c,
    )
else:
    params, loss_mat = training_loop_grad(
        n_epochs = 100,
        learning_rate = lr,
        params = params,
        t_u = t_u2,
        t_c = t_c,
    )
    pass

plt.plot(loss_mat)

In [ ]:
dir(optim)

In [174]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
lr = 1e-1
optimiser = optim.SGD([params], lr=lr)

In [ ]:
epochs=100

for epoch in range(epochs):

    t_p = model(t_u2, *params)
    loss = lossFn(t_p, t_c)

    optimiser.zero_grad() # initialise optimiser gradient SUM = 0

    loss.backward() # run the backward loop
    optimiser.step() # step the optimiser based on the current grad and lr 

    print('Epoch %d, Loss %f' % (epoch+1, float(loss)))

params

## shuffling the dataset

In [175]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:] # -val_index to end

shuffled_indices, train_indices, val_indices

(tensor([ 0,  7, 10,  1,  6,  9,  3,  2,  4,  8,  5]),
 tensor([ 0,  7, 10,  1,  6,  9,  3,  2,  4]),
 tensor([8, 5]))

In [176]:
val_t_u = t_u[val_indices]
val_t_c = t_c[val_indices]

train_t_u = t_u[train_indices]
train_t_c = t_c[train_indices]

train_t_un = (train_t_u - train_t_u.mean()) / train_t_u.std()
val_t_un = (val_t_u - val_t_u.mean()) / val_t_u.std()

epochs = 100

for epoch in range(epochs):

    train_t_p = model(train_t_u, *params)
    train_loss = lossFn(train_t_p, train_t_u)

    val_t_p = model(val_t_u, *params)
    val_loss = lossFn(val_t_p, val_t_c)

    optimiser.zero_grad()
    train_loss.backward()
    optimiser.step()

params



tensor([1., 0.], requires_grad=True)